In [1]:
import numpy as np
import pandas as pd
import sys
import os
import glob
import cv2
from tqdm import tqdm

In [17]:
video_files_train = glob.glob('../../kinetics-dataset/k400/train/*.mp4')

In [37]:
count = 0
for x in video_files_train:
    if x.split('/')[-1].split('.')[0][0] == '_':
        print(x)
        count += 1
        if count == 5:
            break

../../kinetics-dataset/k400/train/_imKC-gBF-I_000164_000174.mp4
../../kinetics-dataset/k400/train/_Uw5azl9Bwg_000010_000020.mp4
../../kinetics-dataset/k400/train/_nBywBbELWA_000016_000026.mp4
../../kinetics-dataset/k400/train/_555H6P8IAY_000523_000533.mp4
../../kinetics-dataset/k400/train/_3-RIujVltk_000651_000661.mp4


In [14]:
'_'.join(video_files_train[0].split('/')[-1].split('.')[0].split('_')[:2])

'fE7oW_Xu2kE'

In [73]:
def extract_in_sequence(data):
    video_path, path = data[0], data[1]
    # TODO: If it fails as the value does not meet minimum frames criteria, then just produce 20 frames in sequence.
    cap = cv2.VideoCapture(video_path)
    # frame_holder = np.zeros((20, 128, 128, 3))
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if ret and count < 20:
            frame = cv2.resize(frame, (128, 128))
            cv2.imwrite(f'{path}{count}.jpg', frame)
            # frame_holder[count] = frame
            count += 1
        else:
            break
    cap.release()

In [74]:
def extract_in_probs(data):
    video_path, path = data
    # TODO: Generate probability of values and get those specific frames only.
    # random_selector = np.random.randint(20, 60, 20)
    x1, x2 = 20, 60
    random_selector = np.arange(x1, x2, (x2-x1)/20).astype(int)
    cap = cv2.VideoCapture(video_path)
    frame_holder = np.zeros((20, 128, 128, 3))
    frame_number = []
    count, valid_selector = 0, 0
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            if count in random_selector:
                frame = cv2.resize(frame, (128, 128))
                frame_number.append(count)
                frame_holder[valid_selector] = frame
                valid_selector += 1
            count += 1
        else:
            break
    cap.release()
    if valid_selector < 19:
        extract_in_sequence([video_path, path])
    else:
        for x in range(20):
            cv2.imwrite(f'{path}{frame_number[x]}.jpg', frame_holder[x])

In [75]:
def extract_frames(data):
    try:
        video_path, target_dir = data[0], data[1]
        cap = cv2.VideoCapture(video_path)
        if cap.isOpened() == False:
            return f'Video is not opened: {video_path}'
        else:
            del cap
            file_name = '_'.join(video_path.split('/')[-1].split('.')[0].split('_')[:-2])
            target_name = target_dir + file_name + '/'
            # print(target_name)
            if os.path.exists(target_name) == False:
                os.makedirs(target_name)
            extract_in_probs([video_path, target_name])
    except:
        return f'Error: {data[0]}'

In [77]:
# extract_frames([video_files_train[0], '../../kinetics_frames_all/'])
for x in range(2):
    extract_frames([video_files_train[x], '../../kinetics_frames_all/'])